In [3]:
#! /usr/bin/env python3

import torch
from torch import nn
import blobconverter
from onnxsim import simplify
import onnx
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1) 

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        return X
    # def forward(self, image):
    #     return torch.rand(1, dim=0)


shape = (10,4)
model = Model()
X = torch.ones(shape, dtype=torch.float32)


onnx_file = "../../results/test.onnx"


torch.onnx.export(
    model,
    X,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
)


onnx_simplified_path = "../../results/onnx_simple.onnx"
onnx_model =  onnx.load(onnx_file)
model_simp, check = simplify(onnx_model)
onnx.save(model_simp, onnx_simplified_path)


blobconverter.from_onnx(
    model=onnx_file,
    data_type="FP16",
    shaves=6,
    use_cache=False,
    output_dir="../../results",
    optimizer_params=[]
)

[==================================================]
Done


PosixPath('../../results/test_openvino_2021.4_6shave.blob')